# Social Media Sentiment Analysis Project
-> Problem Statement (Final Step After Conclusion - Rehab Responsibility)

---------------

## Step 1: Reading and Understanding the data.
Assigned to: Akayiz/Youstina

------

## Step 2: Data Cleaning and Preprocessing
Assigned to: David/Zedan

-----

## Step 3: Exploratory Data Analysis
Assigned to: Amany/Rehab

------

## Step 4: Premodelling Phase

-------

## Step 5: Modelling

-----

## Conclusion